# ***Importing Necessary Libraries***

In [1]:
import pandas as pd
import numpy as np
import sys

# **Prediction Class**

In [2]:
class Prediction:

  def __init__(self, dir_to_data, dir_to_thetas_metadata):
    
    #read  input data and remove the example (row) whose values are not provided or contains NA.
    self.data=pd.read_csv(dir_to_data).dropna()
    
    #list for storing thetas of each layer
    self.thetas=[]

    #read meta data from the file in the same order as it was written to the file
    with open(dir_to_thetas_metadata, 'rb') as file:
      
      #read the total number of layers in the network
      self.no_of_layers = np.load(file)

      #read and use corresponding activation function
      activation_fnt = np.load(file, allow_pickle=True)
      if activation_fnt == 'Sigmoid':
        self.ActivationFnt = self.SigmoidFnt
      else:
        self.ActivationFnt = self.ReLU

      #read thetas
      for i in range(self.no_of_layers-1):
        self.thetas.append(np.load(file))
      
      #read column names which needs to be removed
      remove_cols = np.load(file)
      
      #remove columns not required in the features
      if len(remove_cols):
        self.data=self.data.drop(remove_cols, axis=1)

      #read data related to scaling
      self.scale=np.load(file)
      self.scale_type=np.load(file, allow_pickle=True)

      #read minimium and maximum values of the dataset used for creating model
      self.min=np.load(file, allow_pickle=True)
      self.max=np.load(file, allow_pickle=True)
  
    #if number of columns of the input data (features) is not equal to the nodes in the input layer
    if self.data.shape[1] != self.thetas[0].shape[0] -1:
      sys.exit(f'Features provided are either more or less than input nodes. Please check your input data file.') 


  #Perform scaling of the features
  def scaleDataset(self):

    if self.scale_type: #between 0 and 1
      self.data = self.data.apply(lambda x: (x-self.min)/(self.max-self.min), axis=1)
    
    else: #between -1 and 1
      self.data = self.data.apply(lambda x: 2*((x-self.min)/(self.max-self.min))-1, axis=1)


  #Add bias to the activations as column vector
  def AddBias(self, X):
    return np.c_[ np.ones(X.shape[0]), X]

  #Apply linear function
  def LinearFnt(self, X, theta):
    return np.dot(X, theta)

  #Apply Sigmoid function
  def SigmoidFnt(self, Z):
    return 1/(1+(np.exp(-Z)))

  #Apply ReLU
  def ReLU(self, Z):
    return np.maximum(0, Z)

  #perform prediction on input data
  def prediction(self):
    
    #scale the dataset
    if self.scale:
      self.scaleDataset()
    
    self.data =self.data.to_numpy()

    #Forward Propagation
    for i in range(self.no_of_layers):

      if i == 0: #input layer
        activations=self.AddBias(self.data)

      elif i == self.no_of_layers -1: #output layer
        output=self.LinearFnt(activations, self.thetas[i-1]) #no activation function

      else:
        activations =self.AddBias(self.ActivationFnt(self.LinearFnt(activations, self.thetas[i-1])))

    print(f'The Predicted Values for the given inputs is: \n{output}')
    return output


# **Output**

# *Worldwide*

In [3]:
#directories to the data
dir_to_data_world ='/content/World_Data_Template_For_Prediction_Code.csv'
dir_to_thetas_metadata_world='/content/Optimal_thetas_of Neural_Network_with_Metadata_Worldwide.npy'

In [4]:
pred_world=Prediction(dir_to_data_world, dir_to_thetas_metadata_world)
_=pred_world.prediction()

The Predicted Values for the given inputs is: 
[[191208.13486694]
 [191208.13486694]
 [191208.13486694]
 [191208.13486694]
 [191208.13486694]]


## *US*

In [5]:
#directories to the data
dir_to_data_US ='/content/US_Data_Template_For_Prediction_Code.csv'
dir_to_thetas_metadata_US='/content/Optimal_thetas_of Neural_Network_with_Metadata_US.npy'

In [6]:
pred_US=Prediction(dir_to_data_US, dir_to_thetas_metadata_US)
_=pred_US.prediction()

The Predicted Values for the given inputs is: 
[[23009.74989366]
 [23009.74989392]
 [23009.74989364]
 [23009.7498939 ]
 [23009.74989383]
 [23009.74989384]]
